In [16]:
import numpy as np

# mne imports
import mne
from mne import io, read_epochs, concatenate_epochs
from mne.datasets import sample
from mne import Epochs, create_info, events_from_annotations
from mne.datasets import eegbci
from mne.decoding import CSP
from mne.time_frequency import AverageTFR

# EEGNet-specific imports
from EEGModels import EEGNet
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K

# PyRiemann imports
from pyriemann.estimation import XdawnCovariances
from pyriemann.tangentspace import TangentSpace
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
# tools for plotting confusion matrices
from matplotlib import pyplot as plt

In [140]:
cd epochs

[Errno 2] No such file or directory: 'epochs'
/Users/jang-wonjun/Documents/eeg/ml/epochs


In [141]:
ls

epoch0-epo.fif  epoch2-epo.fif  epoch4-epo.fif
epoch1-epo.fif  epoch3-epo.fif  epoch5-epo.fif


In [142]:
n_epochs = 6
epoch_list = [f'epoch{i}-epo.fif' for i in range(n_epochs)]

In [143]:
epoch = concatenate_epochs([read_epochs(epoch_i) for epoch_i in epoch_list])
epoch

Reading /Users/jang-wonjun/Documents/eeg/ml/epochs/epoch0-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    2000.00 ms
        0 CTF compensation matrices available
Not setting metadata
10 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/jang-wonjun/Documents/eeg/ml/epochs/epoch1-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    2000.00 ms
        0 CTF compensation matrices available
Not setting metadata
10 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/jang-wonjun/Documents/eeg/ml/epochs/epoch2-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    2000.00 ms
        0 CTF compensation matrices available
Not setting metadata
10 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/jang-wonjun/Documents/eeg/ml/epochs/epoch3-epo.fif ...
    Found the data of interest:
        t =  

Number of events,60
Events,finger_movement: 60
Time range,-2.000 – 2.000 s
Baseline,off


In [144]:
fmin, fmax = 8, 20
epoch_filter = epoch.copy().filter(fmin,fmax)

Setting up band-pass filter from 8 - 20 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 20.00 Hz
- Upper transition bandwidth: 5.00 Hz (-6 dB cutoff frequency: 22.50 Hz)
- Filter length: 213 samples (1.664 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 420 out of 420 | elapsed:    0.1s finished


In [145]:
w_tmin, w_tmax = 0, 1.5
on_event = epoch_filter.copy().crop(w_tmin,w_tmax)
off_event = epoch_filter.copy().crop(-2+w_tmin,-2+w_tmax)
        
X = np.concatenate([on_event.get_data(),off_event.get_data()],axis=0)*1000
y = [1 for i in range(X.shape[0]//2)]+[0 for i in range(X.shape[0]//2)]

In [146]:
X.shape # (trials,channels,samples)

(120, 7, 193)

In [147]:
kernels, chans, samples = 1, 7, 193

In [148]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=42)


In [129]:
X_val,X_test,y_val,y_test = train_test_split(X_test, y_test,test_size=0.5, random_state=42)

In [149]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [150]:
model = EEGNet(nb_classes = 2, Chans = chans, Samples = samples, 
               dropoutRate = 0.5, kernLength = 32, F1 = 8, D = 2, F2 = 16, 
               dropoutType = 'Dropout')

model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics = ['accuracy'])

In [151]:
numParams    = model.count_params()

In [152]:
checkpointer = ModelCheckpoint(filepath='/tmp/checkpoint.h5', verbose=1,
                               save_best_only=True)

In [153]:
class_weights = {0:1, 1:1}

In [154]:
fittedModel = model.fit(X_train, y_train, batch_size = 16, epochs = 300, 
                        verbose = 2, validation_data=(X_test, y_test),
                        callbacks=[checkpointer], class_weight = class_weights)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.69332, saving model to /tmp/checkpoint.h5
6/6 - 0s - loss: 0.7138 - accuracy: 0.3542 - val_loss: 0.6933 - val_accuracy: 0.4583 - 314ms/epoch - 52ms/step
Epoch 2/300

Epoch 2: val_loss did not improve from 0.69332
6/6 - 0s - loss: 0.6971 - accuracy: 0.5208 - val_loss: 0.6933 - val_accuracy: 0.4583 - 38ms/epoch - 6ms/step
Epoch 3/300

Epoch 3: val_loss improved from 0.69332 to 0.69323, saving model to /tmp/checkpoint.h5
6/6 - 0s - loss: 0.6845 - accuracy: 0.4896 - val_loss: 0.6932 - val_accuracy: 0.4583 - 51ms/epoch - 9ms/step
Epoch 4/300

Epoch 4: val_loss improved from 0.69323 to 0.69316, saving model to /tmp/checkpoint.h5
6/6 - 0s - loss: 0.6794 - accuracy: 0.5312 - val_loss: 0.6932 - val_accuracy: 0.4583 - 53ms/epoch - 9ms/step
Epoch 5/300

Epoch 5: val_loss improved from 0.69316 to 0.69314, saving model to /tmp/checkpoint.h5


/opt/homebrew/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


6/6 - 0s - loss: 0.6980 - accuracy: 0.5312 - val_loss: 0.6931 - val_accuracy: 0.5417 - 53ms/epoch - 9ms/step
Epoch 6/300

Epoch 6: val_loss did not improve from 0.69314
6/6 - 0s - loss: 0.6831 - accuracy: 0.5521 - val_loss: 0.6932 - val_accuracy: 0.4583 - 41ms/epoch - 7ms/step
Epoch 7/300

Epoch 7: val_loss improved from 0.69314 to 0.69308, saving model to /tmp/checkpoint.h5
6/6 - 0s - loss: 0.6896 - accuracy: 0.5417 - val_loss: 0.6931 - val_accuracy: 0.5417 - 53ms/epoch - 9ms/step
Epoch 8/300

Epoch 8: val_loss improved from 0.69308 to 0.69294, saving model to /tmp/checkpoint.h5
6/6 - 0s - loss: 0.6848 - accuracy: 0.5208 - val_loss: 0.6929 - val_accuracy: 0.5417 - 53ms/epoch - 9ms/step
Epoch 9/300

Epoch 9: val_loss improved from 0.69294 to 0.69282, saving model to /tmp/checkpoint.h5
6/6 - 0s - loss: 0.6799 - accuracy: 0.5417 - val_loss: 0.6928 - val_accuracy: 0.5417 - 52ms/epoch - 9ms/step
Epoch 10/300

Epoch 10: val_loss improved from 0.69282 to 0.69267, saving model to /tmp/checkpo


Epoch 46: val_loss improved from 0.69046 to 0.69041, saving model to /tmp/checkpoint.h5
6/6 - 0s - loss: 0.6484 - accuracy: 0.5833 - val_loss: 0.6904 - val_accuracy: 0.5417 - 53ms/epoch - 9ms/step
Epoch 47/300

Epoch 47: val_loss improved from 0.69041 to 0.69022, saving model to /tmp/checkpoint.h5
6/6 - 0s - loss: 0.6435 - accuracy: 0.6562 - val_loss: 0.6902 - val_accuracy: 0.5417 - 52ms/epoch - 9ms/step
Epoch 48/300

Epoch 48: val_loss improved from 0.69022 to 0.69003, saving model to /tmp/checkpoint.h5
6/6 - 0s - loss: 0.6289 - accuracy: 0.7188 - val_loss: 0.6900 - val_accuracy: 0.5417 - 52ms/epoch - 9ms/step
Epoch 49/300

Epoch 49: val_loss improved from 0.69003 to 0.68992, saving model to /tmp/checkpoint.h5
6/6 - 0s - loss: 0.6495 - accuracy: 0.6458 - val_loss: 0.6899 - val_accuracy: 0.5417 - 52ms/epoch - 9ms/step
Epoch 50/300

Epoch 50: val_loss improved from 0.68992 to 0.68985, saving model to /tmp/checkpoint.h5
6/6 - 0s - loss: 0.6367 - accuracy: 0.6667 - val_loss: 0.6899 - val

Epoch 88/300

Epoch 88: val_loss improved from 0.68798 to 0.68795, saving model to /tmp/checkpoint.h5
6/6 - 0s - loss: 0.5901 - accuracy: 0.7604 - val_loss: 0.6879 - val_accuracy: 0.5417 - 54ms/epoch - 9ms/step
Epoch 89/300

Epoch 89: val_loss did not improve from 0.68795
6/6 - 0s - loss: 0.6234 - accuracy: 0.6875 - val_loss: 0.6884 - val_accuracy: 0.5417 - 39ms/epoch - 6ms/step
Epoch 90/300

Epoch 90: val_loss did not improve from 0.68795
6/6 - 0s - loss: 0.5999 - accuracy: 0.6875 - val_loss: 0.6886 - val_accuracy: 0.5417 - 37ms/epoch - 6ms/step
Epoch 91/300

Epoch 91: val_loss did not improve from 0.68795
6/6 - 0s - loss: 0.6027 - accuracy: 0.6667 - val_loss: 0.6886 - val_accuracy: 0.5417 - 41ms/epoch - 7ms/step
Epoch 92/300

Epoch 92: val_loss did not improve from 0.68795
6/6 - 0s - loss: 0.5740 - accuracy: 0.7708 - val_loss: 0.6886 - val_accuracy: 0.5833 - 39ms/epoch - 7ms/step
Epoch 93/300

Epoch 93: val_loss did not improve from 0.68795
6/6 - 0s - loss: 0.6077 - accuracy: 0.6771 

Epoch 135/300

Epoch 135: val_loss improved from 0.68065 to 0.67812, saving model to /tmp/checkpoint.h5
6/6 - 0s - loss: 0.5771 - accuracy: 0.7500 - val_loss: 0.6781 - val_accuracy: 0.5000 - 53ms/epoch - 9ms/step
Epoch 136/300

Epoch 136: val_loss improved from 0.67812 to 0.67639, saving model to /tmp/checkpoint.h5
6/6 - 0s - loss: 0.5367 - accuracy: 0.7917 - val_loss: 0.6764 - val_accuracy: 0.4583 - 53ms/epoch - 9ms/step
Epoch 137/300

Epoch 137: val_loss improved from 0.67639 to 0.67482, saving model to /tmp/checkpoint.h5
6/6 - 0s - loss: 0.5441 - accuracy: 0.7292 - val_loss: 0.6748 - val_accuracy: 0.4583 - 53ms/epoch - 9ms/step
Epoch 138/300

Epoch 138: val_loss did not improve from 0.67482
6/6 - 0s - loss: 0.5597 - accuracy: 0.7188 - val_loss: 0.6757 - val_accuracy: 0.4583 - 38ms/epoch - 6ms/step
Epoch 139/300

Epoch 139: val_loss did not improve from 0.67482
6/6 - 0s - loss: 0.5701 - accuracy: 0.6979 - val_loss: 0.6751 - val_accuracy: 0.4583 - 38ms/epoch - 6ms/step
Epoch 140/300




Epoch 180: val_loss did not improve from 0.66708
6/6 - 0s - loss: 0.5595 - accuracy: 0.7396 - val_loss: 0.6854 - val_accuracy: 0.5417 - 38ms/epoch - 6ms/step
Epoch 181/300

Epoch 181: val_loss did not improve from 0.66708
6/6 - 0s - loss: 0.5306 - accuracy: 0.8438 - val_loss: 0.6827 - val_accuracy: 0.5417 - 38ms/epoch - 6ms/step
Epoch 182/300

Epoch 182: val_loss did not improve from 0.66708
6/6 - 0s - loss: 0.5192 - accuracy: 0.8125 - val_loss: 0.6791 - val_accuracy: 0.5000 - 37ms/epoch - 6ms/step
Epoch 183/300

Epoch 183: val_loss did not improve from 0.66708
6/6 - 0s - loss: 0.5463 - accuracy: 0.7708 - val_loss: 0.6740 - val_accuracy: 0.5417 - 38ms/epoch - 6ms/step
Epoch 184/300

Epoch 184: val_loss did not improve from 0.66708
6/6 - 0s - loss: 0.5114 - accuracy: 0.7708 - val_loss: 0.6695 - val_accuracy: 0.5000 - 38ms/epoch - 6ms/step
Epoch 185/300

Epoch 185: val_loss did not improve from 0.66708
6/6 - 0s - loss: 0.5280 - accuracy: 0.7396 - val_loss: 0.6700 - val_accuracy: 0.4583 

Epoch 228/300

Epoch 228: val_loss did not improve from 0.66708
6/6 - 0s - loss: 0.4868 - accuracy: 0.8125 - val_loss: 0.7110 - val_accuracy: 0.5000 - 39ms/epoch - 7ms/step
Epoch 229/300

Epoch 229: val_loss did not improve from 0.66708
6/6 - 0s - loss: 0.4879 - accuracy: 0.7812 - val_loss: 0.7057 - val_accuracy: 0.5000 - 38ms/epoch - 6ms/step
Epoch 230/300

Epoch 230: val_loss did not improve from 0.66708
6/6 - 0s - loss: 0.4944 - accuracy: 0.7812 - val_loss: 0.7026 - val_accuracy: 0.4583 - 38ms/epoch - 6ms/step
Epoch 231/300

Epoch 231: val_loss did not improve from 0.66708
6/6 - 0s - loss: 0.5246 - accuracy: 0.6667 - val_loss: 0.6954 - val_accuracy: 0.4583 - 39ms/epoch - 6ms/step
Epoch 232/300

Epoch 232: val_loss did not improve from 0.66708
6/6 - 0s - loss: 0.4779 - accuracy: 0.8229 - val_loss: 0.6899 - val_accuracy: 0.4583 - 39ms/epoch - 6ms/step
Epoch 233/300

Epoch 233: val_loss did not improve from 0.66708
6/6 - 0s - loss: 0.5618 - accuracy: 0.6562 - val_loss: 0.6888 - val_acc

6/6 - 0s - loss: 0.4901 - accuracy: 0.8021 - val_loss: 0.7014 - val_accuracy: 0.5417 - 39ms/epoch - 6ms/step
Epoch 276/300

Epoch 276: val_loss did not improve from 0.66708
6/6 - 0s - loss: 0.5058 - accuracy: 0.7917 - val_loss: 0.7010 - val_accuracy: 0.5000 - 39ms/epoch - 6ms/step
Epoch 277/300

Epoch 277: val_loss did not improve from 0.66708
6/6 - 0s - loss: 0.4706 - accuracy: 0.7292 - val_loss: 0.6965 - val_accuracy: 0.5000 - 36ms/epoch - 6ms/step
Epoch 278/300

Epoch 278: val_loss did not improve from 0.66708
6/6 - 0s - loss: 0.4708 - accuracy: 0.8229 - val_loss: 0.6921 - val_accuracy: 0.4583 - 38ms/epoch - 6ms/step
Epoch 279/300

Epoch 279: val_loss did not improve from 0.66708
6/6 - 0s - loss: 0.4375 - accuracy: 0.8021 - val_loss: 0.6933 - val_accuracy: 0.5000 - 39ms/epoch - 6ms/step
Epoch 280/300

Epoch 280: val_loss did not improve from 0.66708
6/6 - 0s - loss: 0.4691 - accuracy: 0.8229 - val_loss: 0.6927 - val_accuracy: 0.5000 - 37ms/epoch - 6ms/step
Epoch 281/300

Epoch 281: 

In [155]:
# load optimal weights
model.load_weights('/tmp/checkpoint.h5')

In [156]:
probs       = model.predict(X_test)
preds       = probs.argmax(axis = -1)  
acc         = np.mean(preds == y_test.argmax(axis=-1))
print("Classification accuracy: %f " % (acc))

1/1 [==============================] - 0s 55ms/step
Classification accuracy: 0.541667 


In [84]:
probs       = model.predict(X_test)
acc         = np.mean(probs == y_test)
print("Classification accuracy: %f " % (acc))

1/1 [==============================] - 0s 17ms/step
Classification accuracy: 0.583333 


In [34]:
csp = CSP(n_components=10, reg=None, log=True, norm_trace=False)
csp.fit_transform(X,y).shape

Computing rank from data with rank=None
    Using tolerance 0.0016 (2.2e-16 eps * 7 dim * 1e+12  max singular value)
    Estimated rank (mag): 7
    MAG: rank 7 computed from 7 data channels with 0 projectors
Reducing data rank from 7 -> 7
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 0.0012 (2.2e-16 eps * 7 dim * 7.8e+11  max singular value)
    Estimated rank (mag): 7
    MAG: rank 7 computed from 7 data channels with 0 projectors
Reducing data rank from 7 -> 7
Estimating covariance using EMPIRICAL
Done.


(120, 7)